In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.chat_models.openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

In [13]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [14]:
# Carregar os modelos OpenAI - Embedding e Chat

embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=300)

In [15]:
pdf_link = "lol.pdf"

loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [16]:
# Separar em Chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 4000,
    chunk_overlap = 20,
    length_function = len,
    add_start_index = True
)

chunks = text_splitter.split_documents(pages)

In [17]:
# Salvar os chunks no vectorDB

vectordb = Chroma(embedding_function=embeddings_model, persist_directory="rerankdb")

In [18]:
# Carregar o DB

naive_retriever = vectordb.as_retriever(search_kwargs={"k": 10})

In [19]:
os.environ["COHERE_API_KEY"] = ""

In [26]:
rerank = CohereRerank(top_n=3, model="command")

compressor_retriever = ContextualCompressionRetriever(
    base_compressor=rerank,
    base_retriever=naive_retriever
)

In [27]:
TEMPLATE = """"
    Você é um especialista em league of legends. Responda a pergunta abaixo utilizando o contexto informado.

    Query:
    {question}

    Context:
    {context}
"""

rag_promp = ChatPromptTemplate.from_template(TEMPLATE)

In [29]:
setup_retrieval = RunnableParallel({
    "question": RunnablePassthrough(),
    "context": compressor_retriever
})

output_parser = StrOutputParser()

compressor_retrieval_chain =  setup_retrieval | rag_promp | llm | output_parser

In [30]:
compressor_retrieval_chain.invoke("Como posso subir rapidamente de elo?")

'Para subir rapidamente de elo em League of Legends, é importante focar em algumas estratégias-chave. Primeiramente, é fundamental dominar um pequeno pool de campeões e se aprimorar constantemente com eles, conhecendo suas mecânicas e matchups. Além disso, é importante se comunicar de forma eficaz com sua equipe, seja através de pings ou chat, para coordenar jogadas e objetivos. Outro ponto crucial é manter a mentalidade positiva e focar em seu próprio desempenho, buscando sempre aprender com as derrotas e melhorar a cada partida. Por fim, dedicar tempo para estudar o jogo, assistir streams de jogadores de alto nível e participar de comunidades online pode ajudar a aprimorar suas habilidades e estratégias.'